In [ ]:
import snowflake
from snowflake.snowpark.context import get_active_session

session = get_active_session()

import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
import pandas as pd


source_data = ['terraclimate']
terraclimate_variables = ['aet', 'pet', 'q', 'ppt', 'soil', 'tmax', 'tmin', 'vpd']


for i in range(len(source_data)):
    folder_name = source_data[i]
    print(folder_name)
    source_dir = Path(f"./data/{folder_name}/")  

    csv_files = sorted(source_dir.glob("*.csv"))

    #training_df = pd.DataFrame()
    training_df = pd.read_csv('data/water_quality_training_dataset.csv')

    for col in training_df.columns:
        training_df = training_df.rename(columns={col: col.lower()})

    training_df['sample date'] = pd.to_datetime(training_df['sample date'],format='%d-%m-%Y')

    training_df.head(5)
    
    submission_df = pd.read_csv('data/submission_template.csv')

    for col in submission_df.columns:
        submission_df = submission_df.rename(columns={col: col.lower()})

    submission_df['sample date'] = pd.to_datetime(submission_df['sample date'],format='%d-%m-%Y')
    
    

            

In [ ]:
for i in range(len(csv_files)):

#for i in range(3):

    print(csv_files[i])

    temp_df = pd.read_csv(f"{csv_files[i]}")

    for col in temp_df.columns:
        temp_df = temp_df.rename(columns={col: col.lower()})

    try:
        temp_df['sample date'] = pd.to_datetime(temp_df['sample date'],format='%d-%m-%Y')
    except:
        temp_df['sample date'] = pd.to_datetime(temp_df['sample date'],format='%d/%m/%Y')
    
    #print(temp_df.shape)
    
    if 'training' in str(csv_files[i]):

        training_df = training_df.merge(temp_df, on=['latitude', 'longitude', 'sample date'], how='inner')
        
    elif 'validation' in str(csv_files[i]):
        
        submission_df = submission_df.merge(temp_df, on=['latitude', 'longitude', 'sample date'], how='inner') 

    print(training_df.shape)
    print(submission_df.shape)

In [ ]:
training_df.shape

In [ ]:
training_df.head(5)

In [ ]:
submission_df.head(5)

In [ ]:
training_df.to_csv('terraclimate_features_training_combined.csv', index=False)
submission_df.to_csv('terraclimate_features_validation_combined.csv', index=False)

In [ ]:
session.sql("""
    PUT file://terraclimate_features_training_combined.csv
    'snow://workspace/USER$.PUBLIC."EY-AI-and-Data-Challenge-version2"/versions/live/data/'
    AUTO_COMPRESS=FALSE
    OVERWRITE=TRUE
""").collect()

print("File saved! Refresh the browser to see the files in the sidebar")

In [ ]:
session.sql("""
    PUT file://terraclimate_features_validation_combined.csv
    'snow://workspace/USER$.PUBLIC."EY-AI-and-Data-Challenge-version2"/versions/live/data/'
    AUTO_COMPRESS=FALSE
    OVERWRITE=TRUE
""").collect()

print("File saved! Refresh the browser to see the files in the sidebar")